In [ ]:
import sqlite3
import glob
import urllib.parse
import records
import requests
from bs4 import BeautifulSoup
import MeCab as mc
import re
from collections import Counter
from wordcloud import WordCloud
from matplotlib import pyplot as plt
import json

# import sqlite3litedatastore as datastore

In [ ]:
conn = None

# データベースに接続する
def connect():
    global conn
    conn = sqlite3.connect("./sample.db")
    
# コネクションを断つ
def close():
    conn.close()
    
# テーブル作成
def create_table():
    # DROPは消す.
    conn.execute("DROP TABLE IF EXISTS countries")
    conn.execute("""CREATE TABLE countries (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            country_name TEXT,
            content TEXT
    )""")
    
# loadは積むという意味
def load(name, content):
    conn.execute(
        "INSERT INTO Countries (country_name, content) VALUES (?,?)", (name, content))
    conn.commit()
    
def get(doc_id, fl):
    row_ls = conn.execute(
        "SELECT {} FROM docs WHERE id = ?".format(",".join(fl)),(doc_id,)).fetchone()
    row_dict = {}
    for key, value in zip(fl, row_ls):
        row_dict[key] = value
    return row_dict

def get_all_ids(limit, offset=0):
    return [record[0] for record in conn.execute("SELECT id FROM docs LIMIT ? OFFSET ?",(limit, offset))]

In [ ]:
url = "https://ja.wikipedia.org"
r = requests.get(url+"/wiki/国の一覧")

html_soup = BeautifulSoup(r.text, "html.parser")
table = html_soup.find("table",class_= "wikitable")
connect()
create_table()

for tr in table.find_all("tr"):
    link = tr.find_all("a")
    if link:
        country_name = link[1].text
        country_link = link[1].get("href")
        
        res = requests.get(url+country_link)
        load(country_name,res.text)
close()

In [ ]:
con = None
con = sqlite3.connect("./sample.db")
cursor = con.cursor()
map_in=input()
res = cursor.execute("SELECT content FROM Countries WHERE country_name = ?",[map_in])
# print(res)
# bs = BeautifulSoup(cursor.fetchall()[0][0],"html.parser")
bs = BeautifulSoup(cursor.fetchone()[0],"html.parser")

In [ ]:
# print(bs.find("div", id = "content").text)
div_all = bs.find("div", id = "content").text
# print(re.sub("[\n,\xa0^,\<http\S*]","",div_all))
del_n = re.sub("[\n,\xa0^]","",div_all)
del_wiki = re.sub("出典|編集|参照|ページ|cs|閲覧|wiki[pm]edia|parser|CentralStatistiOffice|font|url|commons|alt|svg","",del_n)
del_wiki2 = re.sub("left|background|kern|km|right|emcenter|position|upload|output|thumb|mw|png|inherit|org|px|citation","",del_wiki)
del_wiki3 = re.sub("emcenter|repeat|それ|一覧|略称|以上|こと|表記|もの|記事|以下|ため|直接|出版|構成|ほか|これ|よう","",del_wiki2)
del_all = re.sub("https?\S*\s","",del_wiki3)
del_space = re.sub(" ","",del_all)
# print(del_space)

In [ ]:
# tagger = mc.Tagger("-Ochasen")
tagger = mc.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")

text = "新国立美術館"
node = tagger.parse(text)
print(node)

In [ ]:
def mecab_analysis(texts):
    t = mc.Tagger("")
    t.parse('')
    output = []
    node =  t.parseToNode(texts)  
    while node:
        if node.surface != "":  # ヘッダとフッタを除外
            word_type = node.feature.split(",")[0]
            if word_type in ['名詞', '形容詞']:
                output.append(node.surface)
        node = node.next
        if node is None:
            break
    return output

In [ ]:
text = " ".join(mecab_analysis(del_space))

In [ ]:
fpath = "/home/t-katou/.local/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/ttf/ipaexg.ttf"
wordcloud = WordCloud(
    background_color="white",
    font_path=fpath,
    width=800,
    collocations = False,
    max_words=70,
    height=600).generate(text)

In [ ]:
plt.figure(figsize=(15,12))
plt.imshow(wordcloud)
# wordcloud.to_image("aaa.png")

In [ ]:
import matplotlib as mpl
print(mpl.matplotlib_fname())

In [8]:
Counter(mecab_analysis(del_space))
mecab_analysis(del_space)
# sorted(Counter(mecab_analysis(del_space)).items(),key=lambda x:x[1],reverse=True)

['アイスランド',
 ':',
 'フリー',
 '百科',
 '事典',
 'ウィキペディア',
 'Wikipedia',
 'ナビゲーション',
 '移動',
 '検索',
 '移動',
 'アイスランド',
 '共和',
 '国',
 'Ísland',
 '国旗',
 '）(',
 '国',
 '章',
 ')',
 '国',
 '標語',
 '国歌',
 'Lofsöngur',
 'アイスランド',
 '語',
 '賛美',
 '歌',
 '公用',
 '語',
 'アイスランド',
 '語',
 '首都',
 'レイキャビク',
 '最大',
 '都市',
 'レイキャビク',
 '政府',
 '大統領',
 'グズニ・ヨハンネソン',
 '首相',
 'カトリーン・ヤコブスドッティル',
 '面積',
 '総計',
 '1030002',
 '105',
 '位',
 '水面',
 '積',
 '率',
 '2',
 '.',
 '7',
 '%',
 '人口',
 '総計',
 '2018',
 '年',
 '355620',
 '人',
 '170',
 '位',
 '人口',
 '密度',
 '3',
 '人',
 '/',
 '2',
 'GDP',
 '自国',
 '通貨',
 '表示',
 '合計',
 '2008',
 '年',
 '14650',
 '億',
 '[',
 '1',
 ']',
 'アイスランド',
 '・',
 'クローナ',
 'GDP',
 '(',
 'MER',
 ')',
 '合計',
 '2008',
 '年',
 'xxxxxx',
 'ドル',
 '91',
 '位',
 'GDP',
 '(',
 'PPP',
 ')',
 '合計',
 'xxxx',
 '年',
 'xxxxxx',
 'ドル',
 '（???',
 '位',
 '1',
 '人',
 'あたり',
 '40024',
 '[',
 '1',
 ']',
 'ドル',
 '独立',
 'デンマーク',
 '1944',
 '年',
 '6',
 '月',
 '17',
 '日',
 '通貨',
 'アイスランド',
 '・',
 'クローナ',
 '(',
 'ISK',
 ')',
 '時間',
 '帯',
 'UTC',

In [ ]:
# a = ["い","あ","う"]
# print(" ".join(a))
mecab_sorted=sorted(Counter(mecab_analysis(del_space)).items(),key=lambda x:x[1],reverse=True)
# print(mecab_sorted[0][0])
list = []

for str in mecab_sorted:
    list.append(str[0])
    
# print(list)
rennketu=" ".join(list)
print(rennketu)

In [ ]:
def set_annotation(doc_id, name, value):
    conn.execute("UPDATE docs SET {0} = ? where id = ?".format(name),(json.dumps(value), doc_id))
    conn.commit()
    
def get_annotation(doc_id, name):
    row = conn.excute("SELECT {0} FROM docs WHERE id =?".format(name))

In [ ]:
if "__name__" == "__main__":
    connect()
    values = []
    for filename in glob.glob("./data/wikipedia/*.html"):
        with open(filename) as fin:
            html = fin.read()
            text, title = scrape.scrape(html)
            print("scraped:", title)
            url = "https://ja.wikipedia.org/wiki/{0}".format(urllib.parse.quote(title))
            values.append((text, json.dumps({"url":url, "title":title})))
    load(values)
    
    print(list(datastore.get_all_ids(limit=-1)))
    close()
    

In [ ]:
data = []
doc_ids = []
for doc_id in datatastore.get_all_ids(limit=-1):
    data.append(" ".join(mecab_analysis(del_space)))
    
    doc_ids.append(doc_id)
    
vectorizer = TfidfVectorizer(analyzer="word", max_df = 0.9)
vecs = vectorizer.fit_transform(data)

for doc_id, vec in zip(doc_ids, vecs.toarray()):
    meta_info = json.loads(datastore.get(doc_id, ["meta_info"])["meta_info"])
    title = meta_info["title"]
    print(doc_id, title)
    
    forr w_id, tfidf in sorted(enumerate(vec), key=lambda x: x[1],reverse=True)[:10]:
        lemma = vectorizer.get_feature_names()[w_id]
        print("\t{0:s}: {1:f}".format(lemma, tfidf))

    

In [ ]:
print(json.loads(row["meta_info"]))